**Importing required Packages**

In [ ]:
import math
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
import os
from scipy import ndimage
import tensorflow as tf
import matplotlib.pyplot as plt
np.random.seed(1)

**Getting familiar with our dataset**

In [ ]:
# First Source of data
train_data_1 = pd.read_csv("../input/digit-recognizer/train.csv")
# Second Source of data
train_data_2 = pd.read_csv("../input/mnist-in-csv/mnist_train.csv")
# Third Source of data
train_data_3 = pd.read_csv("../input/mnist-in-csv/mnist_test.csv")
# Combine the data
train_data = pd.DataFrame(np.row_stack((train_data_1,train_data_2, train_data_3)))
#train_data = pd.DataFrame(np.row_stack((train_data_2, train_data_3)))
# Shuffle the data
train_data = train_data.sample(frac=1).reset_index(drop=True)
# Load test data
X_test = pd.read_csv("../input/digit-recognizer/test.csv")

In [ ]:
# Separate into X_train and y_train(target varible)
y_train = train_data[0]
X_train = train_data.drop(0,axis = 1)

In [ ]:
# Shape of X_train
print("Shape of X_train :",X_train.shape)
print("Shape of X_test  :",X_test.shape)
# Shape of y_train
print("Shape of y_train :",y_train.shape)

In [ ]:
# Converting to float values
X_train = X_train.values.astype('float32')
X_test = X_test.values.astype('float32')

In [ ]:
# Each image is a 28x28 pixel image so resulting into 784 pixel values
# Reshape X_train : m * height * width * 1
# One shape dimension can be -1. In this case, the value is inferred from the length of the array and remaining dimensions, i.e. m.
X_train = X_train.reshape([-1, 28, 28,1])
X_test = X_test.reshape([-1, 28, 28,1])
# Shape of X_train
print("Shape of X_train now :",X_train.shape)

In [ ]:
# Example of a picture from dataset
index = 10
plt.imshow(X_train[index,:,:,0],cmap = "gray")
plt.title("y = "+str(y_train[index]))

**Exploratory Data Analysis**

In [ ]:
# Countplot for labels
sns.countplot(train_data[0])
print("Dataset is pretty balanced!")

**CNN Architecture we're going to use**

In [ ]:
from IPython.display import Image
Image(filename = "../input/cnn-architecture/Untitled.png", width=1900, unconfined=True)

In [ ]:
# Normalise input
X_train = X_train/255.0
X_test = X_test/255.0

**Defining Helper Functions to use later**

In [ ]:
# Helper function for one-hot encoding of target variable
# one_hot encoder
# 0 -> [1 0 0 0 0 0 0 0 0 0]
# 1 -> [0 1 0 0 0 0 0 0 0 0]
# ...
def onehot_encoder(label, k):
    """
    Arguments:
    labels -- labels of images(target variable)
    k      -- number of classes
    Returns:
    label_one_hot -- numpy matrix with one-hot encoded labels of shape (m,k)
    """
    m = label.shape[0] # number of training examples
    label_one_hot = np.zeros((m, k))
    for i in range(m):
        label_one_hot[i][int(label[i])] = 1
    return label_one_hot

# Helper function for creating placeholders
def create_placeholders(n_H0, n_W0, n_C0, n_y):
    """
    Arguments:
    n_H0 -- scalar, height of an input image
    n_W0 -- scalar, width of an input image
    n_C0 -- scalar, number of channels of the input
    n_y -- scalar, number of classes  
    Returns:
    X -- placeholder for the data input, of shape [None, n_H0, n_W0, n_C0] and dtype "float"
    Y -- placeholder for the input labels, of shape [None, n_y] and dtype "float"
    is_training -- boolean value(True/False) to tell if we are working on train set or test/dev set. Useful for BatchNorm layer
    """
    X = tf.placeholder(tf.float32,[None,n_H0,n_W0,n_C0])
    Y = tf.placeholder(tf.float32,[None,n_y])
    is_training = tf.placeholder(tf.bool)
    return X, Y, is_training

# Helper function for creating random mini batches
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Arguments:
    X -- input data, of shape (number of examples,28,28,1)
    Y -- true "label" vector of shape (number of examples, number of classes)
    mini_batch_size -- size of the mini-batches, integer
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    np.random.seed(seed)            # for reproducibility
    m = X.shape[0]                  # number of training examples
    mini_batches = []
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation,:,:,:]
    shuffled_Y = Y[permutation,:]
    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size: (k + 1) * mini_batch_size, :, :, :]
        mini_batch_Y = shuffled_Y[k * mini_batch_size: (k + 1) * mini_batch_size, :]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size: m, :, :, :]
        mini_batch_Y = shuffled_Y[num_complete_minibatches * mini_batch_size: m, :]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    return mini_batches

# Helper function for creating sequence mini batches
# Will be used for prediction of test data
def sequence_mini_batches(X, mini_batch_size = 64, seed = 0):
    """
    Arguments:
    X -- input data, of shape (number of examples,28,28,1)
    mini_batch_size -- size of the mini-batches, integer
    Returns:
    mini_batches -- list of synchronous mini_batch_X
    """
    np.random.seed(seed)            # for reproducibility
    m = X.shape[0]                  # number of training examples
    mini_batches = []
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = X[k * mini_batch_size: (k + 1) * mini_batch_size, :, :, :]
        mini_batches.append(mini_batch_X)
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = X[num_complete_minibatches * mini_batch_size: m, :, :, :]
        mini_batches.append(mini_batch_X)
    return mini_batches

# Helper function for initializing parameters
def initialize_parameters():
    """
    Initializes weight parameters to build a neural network with tensorflow. The shapes are:
                        W1 : [5, 5, 1, 32]
                        W2 : [5, 5, 32, 32]
                        W3 : [3, 3, 32, 64]
                        W4 : [3, 3, 64, 64]
    Returns:
    parameters -- a dictionary of tensors containing W1, W2
    """
    tf.set_random_seed(1)                              # for reproducibility
    W1 = tf.get_variable('W1',[5, 5, 1, 32],initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W2 = tf.get_variable('W2',[5, 5, 32, 32],initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W3 = tf.get_variable('W3',[3, 3, 32, 64],initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    W4 = tf.get_variable('W4',[3, 3, 64, 64],initializer = tf.contrib.layers.xavier_initializer(seed = 0))
    parameters = {"W1": W1,
                  "W2": W2,
                  "W3": W3,
                  "W4": W4}
    return parameters

# Helper function for forward propagation
def forward_propagation(X,is_training, parameters):
    """
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "W2"
                  the shapes are given in initialize_parameters
    Returns:
    Z3 -- the output of the last LINEAR unit
    """
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    W2 = parameters['W2']
    W3 = parameters['W3']
    W4 = parameters['W4']
    # CONV2D: stride of 1, padding 'SAME'
    Z1 = tf.nn.conv2d(X,W1,strides = [1,1,1,1],padding = 'SAME')
    # BATCHNORM
    N1 = tf.layers.batch_normalization(Z1, training=is_training)    
    # RELU
    A1 = tf.nn.relu(N1)
    # CONV2D: stride of 1, padding 'SAME'
    Z2 = tf.nn.conv2d(A1,W2,strides = [1,1,1,1],padding = 'SAME')
    # BATCHNORM
    N2 = tf.layers.batch_normalization(Z2, training=is_training)    
    # RELU
    A2 = tf.nn.relu(N2)
    # MAXPOOL: window 2x2, sride 2, padding 'VALID'
    P1 = tf.nn.max_pool(A2,ksize = [1,2,2,1],strides = [1,2,2,1],padding = 'VALID')
    # CONV2D: filters W2, stride 1, padding 'SAME'
    Z3 = tf.nn.conv2d(P1,W3,strides = [1,1,1,1],padding = 'SAME')
    # BATCHNORM
    N3 = tf.layers.batch_normalization(Z3, training=is_training)
    # RELU
    A3 = tf.nn.relu(N3)
    # CONV2D: filters W2, stride 1, padding 'SAME'
    Z4 = tf.nn.conv2d(A3,W4,strides = [1,1,1,1],padding = 'SAME')
    # BATCHNORM
    N4 = tf.layers.batch_normalization(Z4, training=is_training)
    # RELU
    A4 = tf.nn.relu(N4)
    # MAXPOOL: window 2x2, stride 2, padding 'VALID'
    P2 = tf.nn.max_pool(A2,ksize = [1,2,2,1],strides = [1,2,2,1],padding = 'VALID')
    # FLATTEN
    P2 = tf.contrib.layers.flatten(P2)
    # FULLY-CONNECTED with "relu" activation function
    # 256 neurons in output layer.
    Z5 = tf.contrib.layers.fully_connected(P2,num_outputs = 256,activation_fn=tf.nn.relu)
    # FULLY-CONNECTED with non-linear activation function (do not call softmax)
    # 10 neurons in output layer.
    Z6 = tf.contrib.layers.fully_connected(Z5,num_outputs = 10,activation_fn=None)
    return Z6

# Helper function to compute cost
def compute_cost(Z6, Y):
    """
    Arguments:
    Z3 -- output of forward propagation (output of the last LINEAR unit), of shape (10, number of examples)
    Y -- "true" labels vector placeholder, same shape as Z3
    Returns:
    cost - Tensor of the cost function
    """
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=Z6))
    return cost

**Defining the model using our helper functions**

In [ ]:
# Defining the Model
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.009,num_epochs = 100, minibatch_size = 64, print_cost = True):
    """
    Arguments:
    X_train -- training set, of shape (None, 28, 28, 1)
    Y_train -- test set, of shape (None, n_y = 10)
    X_test -- training set, of shape (None, 28, 28, 1)
    Y_test -- test set, of shape (None, n_y = 10)
    learning_rate -- learning rate of the optimization
    num_epochs -- number of epochs of the optimization loop
    minibatch_size -- size of a minibatch
    print_cost -- True to print the cost every 100 epochs
    Returns:
    train_accuracy -- real number, accuracy on the train set (X_train)
    test_accuracy -- real number, testing accuracy on the test set (X_test)
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    tf.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep results consistent (tensorflow seed)
    seed = 3                                          # to keep results consistent (numpy seed)
    (m, n_H0, n_W0, n_C0) = X_train.shape             
    n_y = Y_train.shape[1]                            
    costs = []                                        # To keep track of the cost
    
    # Create Placeholders of the correct shape
    X, Y, is_training = create_placeholders(28,28,1,10)
    # Initialize parameters
    parameters = initialize_parameters()
    # Forward propagation: Build the forward propagation in the tensorflow graph
    Z6 = forward_propagation(X,is_training,parameters)
    # Cost function: Add cost function to tensorflow graph
    cost = compute_cost(Z6,Y)
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer that minimizes the cost.
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    # Initialize all the variables globally
    init = tf.global_variables_initializer()
    # Start the session to compute the tensorflow graph
    config = tf.ConfigProto()
    config.gpu_options.allocator_type = 'BFC'
    with tf.Session(config = config) as sess:
        # Run the initialization
        sess.run(init)
        # Do the training loop
        for epoch in range(num_epochs):
            minibatch_cost = 0.
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)

            for minibatch in minibatches:
                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                # Run the session to execute the optimizer and the cost, the feedict should contain a minibatch for (X,Y).
                _ , temp_cost = sess.run([optimizer,cost],{X:minibatch_X,Y:minibatch_Y, is_training:True})
                
                minibatch_cost += temp_cost / num_minibatches #averaging the cost on 1 mini_batch
                
            # Print the cost every epoch
            if print_cost == True and epoch % 5 == 0:
                print ("Cost after epoch %i: %f" % (epoch, minibatch_cost))
            if print_cost == True and epoch % 1 == 0:
                costs.append(minibatch_cost)
        
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

        # Calculate the correct predictions
        # computational graph
        predict_op = tf.argmax(Z6, 1)
        correct_prediction = tf.equal(predict_op, tf.argmax(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        
        minibatches_train = random_mini_batches(X_train, Y_train, minibatch_size, seed=0)
        train_accuracy_list = []
        for minibatch_train in minibatches_train:
            (minibatch_X_train, minibatch_Y_train) = minibatch_train
            train_accuracy = accuracy.eval(session=sess, feed_dict={X: minibatch_X_train, Y: minibatch_Y_train, is_training:False})
            train_accuracy_list.append(train_accuracy)
        print("Train Accuracy:", sum(train_accuracy_list) / len(train_accuracy_list))

        if Y_test is not None: # While working on dev set
            minibatches_dev  = random_mini_batches(X_test, Y_test, minibatch_size, seed=0)
            dev_accuracy_list   = []
            for minibatch_dev in minibatches_dev:
                (minibatch_X_dev, minibatch_Y_dev)     = minibatch_dev
                dev_accuracy = accuracy.eval(session=sess, feed_dict={X: minibatch_X_dev, Y: minibatch_Y_dev, is_training:False})
                dev_accuracy_list.append(dev_accuracy)
            print("Dev set Accuracy:", sum(dev_accuracy_list) / len(dev_accuracy_list))
            
        else: # While working on submission
            minibatches_test  = sequence_mini_batches(X_test, minibatch_size, seed=0)
            y_pred = []
            for minibatch_test in minibatches_test:
                y_pred_minibatch = list(predict_op.eval(session=sess, feed_dict={X: minibatch_test, is_training:False}))
                for l in range(0,len(y_pred_minibatch)):
                    y_pred.append(y_pred_minibatch[l])
                
            y_pred = np.array(y_pred)
            y_pred_df = pd.DataFrame(y_pred.reshape(-1, 1), index=np.arange(1, 1 + len(y_pred)), columns=["Label"])
            print("Generated predictions!")
            return y_pred_df,parameters
        return parameters

**Time to run the model**

In [ ]:
# One-hot ecoding the labels
y_train_one_hot = onehot_encoder(y_train, 10)

In [ ]:
# Splits data in train and dev set
#X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train_one_hot, test_size = 0.1, random_state=42)
# Run this model while testing
#parameters = model(X_train, y_train, X_dev, y_dev, learning_rate = 0.01,num_epochs = 5, minibatch_size = 64, print_cost = True)

In [ ]:
# Run this model while submitting
y_pred_df,parameters = model(X_train, y_train_one_hot, X_test, Y_test = None, learning_rate = 0.01,num_epochs = 35, minibatch_size = 64, print_cost = True)

In [ ]:
y_pred_df.to_csv("test_predict.csv", index=True, index_label="ImageId")

   **GIVE A THUMBS UP IF YOU LIKE THIS KERNEL**